# Routine.ipynb
---

In this IJulia notebook is presented the partially complete mathematical/statistical modeling routine using **Knowledge Systems**, and later in the future, **Quantum Transport**. A combination of the two components could provide for accurate theoretical (QM-ical) predictions and heuristic (ML) predictions.

## Load the images
---

The image data is located in another folder and separate microstructure images should be made.

In [ ]:
using Images

S1_ = float.(Gray.(load("data/images/S1.JPG")))
S2_ = float.(Gray.(load("data/images/S2.JPG")))
S3_ = float.(Gray.(load("data/images/S3.JPG"))); y, x = size(S1_)

## Prepare the microstructure images
---

Images will be segmented into two parts, the grain and the grain boundaries. For this, a modified thresholding technique will be used. After such, the values will be averaged for the whole ensemble.

In [ ]:
rand_img = [(rand(1:y-300), rand(1:x-300)) for _ in 1:300]

S1_cha = Vector{Vector{Float64}}()
S2_cha = Vector{Vector{Float64}}()
S3_cha = Vector{Vector{Float64}}()

function pushstructtoimg(cha::Vector{Vector{Float64}}, root::Array{Float64, 2})
    for (y, x) in rand_img
        push!(cha, reshape(root[y:y+300, x:x+300], 301^2))
    end
end

pushstructtoimg(S1_cha, S1_)
pushstructtoimg(S2_cha, S2_)
pushstructtoimg(S3_cha, S3_)

## Prepare the external package for additional analyses code
---

Relevant code for strutural analyses is contained in an unpublished Julia package. Also prepare the needed assets for plotting. Here, the `plotly()` backend will be used.

In [ ]:
include("src/Boundaries.jl"); using Boundaries
using Plots; plotly()

xs = ys = [string(i) for i=-150:150]
mdl = cgrad([parse(Colorant, "#BBDEFB"), parse(Colorant, "#2196F3"), parse(Colorant, "#1976D2")])

## Get the ensemble-averaged two-point spatial correlations
---

This is a major part of the routine that is used for modeling the *probability density* of locating grain boundaries within the vicinity of another grain boundary. This assumes, however, that the structure is periodic in both axes.

In [ ]:
S1_ave = Boundaries.ensemble_ave(S1_cha)
S2_ave = Boundaries.ensemble_ave(S2_cha)
S3_ave = Boundaries.ensemble_ave(S3_cha)

S1_ave = Boundaries.convolve_mean(S1_ave)
S2_ave = Boundaries.convolve_mean(S2_ave)
S3_ave = Boundaries.convolve_mean(S3_ave)

### Probability density for first set of fabrication parameters
---
 + $0.05M$ Zn$^{2+}$ with $0.8M$ NaOH solution
 + $100$ dipping cycles
 + $300^{\circ}C$ annealing for 30 mins

In [ ]:
contour(xs, ys, reshape(S1_ave, 301, 301), aspect_ratio=1)

### Probability density for second set of fabrication parameters
---
 + $0.05M$ Zn$^{2+}$ with $0.8M$ NaOH solution
 + $200$ dipping cycles
 + $300^{\circ}C$ annealing for 30 mins

In [ ]:
contour(xs, ys, reshape(S2_ave, 301, 301), aspect_ratio=1)

### Probability density for third set of fabrication parameters
---
 + $0.05M$ Zn$^{2+}$ with $0.8M$ NaOH **heated** solution
 + $100$ dipping cycles
 + $500^{\circ}C$ annealing for 30 mins

In [ ]:
contour(xs, ys, reshape(S3_ave, 301, 301), aspect_ratio=1)

### X-ray diffraction data
---

The following cells show the data for modeling raw data from raw XRD data (*.ASC file extension*).

In [ ]:
using CSV, DataFrames

df1 = CSV.read("data/xrd/S1.ASC", delim=' ', header=["angle", "counts"])
df2 = CSV.read("data/xrd/S2.ASC", delim=' ', header=["angle", "counts"])
df3 = CSV.read("data/xrd/S3.ASC", delim=' ', header=["angle", "counts"])

plot(df1[:angle], [df1[:counts], df2[:counts].+75, df3[:counts].+375])